In [477]:
import pandas as pd
import numpy as np
import sqlalchemy
import sqlite3

In [478]:
df_population = pd.read_csv('fr_population.csv')

In [479]:
df_population = df_population[['Zone','Code zone','Année','Valeur']]
df_population = df_population.rename(columns={'Zone': "pays", 'Code zone':'code_pays', 'Valeur': 'population'})
df_population.head()


,pays,code_pays,Année,population
0,Afghanistan,2,2013,30552
1,Afrique du Sud,202,2013,52776
2,Albanie,3,2013,3173
3,Algérie,4,2013,39208
4,Allemagne,79,2013,82727


In [ ]:
Question15, 16
CLE PRIMAIRE TABLE POPULATION code_pays
CLE PRIMAIRE TABLE dispo_alim code_produit
CLE PRIMAIRE TABLE equilibre_prod code_produit
CLE PRIMAIRE TABLE sous_nutrition code_pays

In [481]:
conn = sqlite3.connect('project3.db')

In [482]:
from sqlalchemy import create_engine

In [483]:
engine = create_engine('sqlite:///:memory:')

In [484]:
df_population.to_sql('population', engine,  index=False, if_exists='replace')

In [486]:

pd.read_sql_table('population', engine)

,pays,code_pays,Année,population
0,Afghanistan,2,2013,30552
1,Afrique du Sud,202,2013,52776
2,Albanie,3,2013,3173
3,Algérie,4,2013,39208
4,Allemagne,79,2013,82727
...,...,...,...,...
170,Venezuela (République bolivarienne du),236,2013,30405
171,Viet Nam,237,2013,91680
172,Yémen,249,2013,24407
173,Zambie,251,2013,14539


In [487]:
 pd.read_sql_query(
"SELECT SUM(population) FROM population WHERE Année=2013", engine)

,SUM(population)
0,8413993


In [488]:

vegetaux = pd.read_csv('fr_vegetaux.csv')
animaux = pd.read_csv('fr_animaux.csv')


In [489]:
vegetaux['nature'] = 'végétal'
animaux['nature'] = 'animal'

In [490]:
products = pd.concat([vegetaux, animaux])


In [491]:

products.rename(columns={
'Code zone' : 'code_pays',
'Zone' : 'pays',
'Code Élément' : 'code_element',
'Élément' : 'element',
'Code Produit' : 'code_produit',
'Produit' : 'produit',
'Code Année': 'code_annee',
'Année' : 'annee',
'Unité' : 'unite',
'Valeur' : 'valeur',
}, inplace=True)

In [492]:
products = products[[
    'code_pays',
    'pays', 
    'code_element', 
    'element', 
    'code_produit', 
    'produit', 
    'annee', 
    'unite', 
    'valeur',
    'nature']]

In [493]:
products = products[products.code_pays!=351]

In [494]:
population = products[products['produit'] == 'Population']
products = products[products['produit'] != 'Population']

In [495]:
population = population.pivot_table(
    index=['code_pays', 'pays', 'annee'],
    columns=['produit'],
    values=['valeur']
).reset_index()



In [496]:
products = products.pivot_table(
    index=['code_pays', 'pays', 'code_produit', 'produit', 'nature', 'annee'],
    columns=['element'],
    values=['valeur'], 
    aggfunc=sum
).reset_index()

In [497]:
products.columns = [
    'code_pays',
    'pays',
    'code_produit',
    'produit',
    'nature',
    'annee',
    'alim_ani',
    'autres_utilisations',
    'dispo_alim_kcal_p_j',
    'dispo_alim_kg_p_an',
    'dispo_mat_gr',
    'dispo_prot',
    'dispo_int',
    'exportation',
    'importation',
    'nourriture',
    'pertes',
    'production',
    'semences',
    'traitement', # transformation ?
    'variation_stock'
]

In [498]:
products = pd.merge(
    products,
    df_population,
)

In [499]:
products['dispo_alim_tonnes'] = (products['dispo_alim_kg_p_an'] * products['population']) / 1000

In [500]:
population.to_sql(
    'population', # nom de la table
    engine, # connexion utilisé (ici, celle donnée par SQLAlchemy)
    index=False,  # on ne veut pas écrire les index dans la table
    if_exists='replace' # si la table existe déjà, on l'écrase
)

In [501]:
dispo_alim = products[[ 'pays', 'code_pays', 'annee', 'produit','code_produit','nature',
          'dispo_alim_tonnes','dispo_alim_kcal_p_j','dispo_prot','dispo_mat_gr'
]]

In [502]:
dispo_alim = dispo_alim.dropna(
    subset=['dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr'],
    how='all'
)

In [503]:
dispo_alim.to_sql('dispo_alim', engine , index=False , if_exists='replace', chunksize=1000)

In [504]:
equilibre_prod = products[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'nature', 'dispo_int', 
                   'alim_ani', 'semences', 'pertes', 'traitement', 'nourriture', 'autres_utilisations']]

In [505]:
equilibre_prod = equilibre_prod.dropna(
    subset=['dispo_int', 'alim_ani', 'semences', 'pertes', 'traitement', 'nourriture', 'autres_utilisations'],
    how='all'
)

In [506]:
equilibre_prod.to_sql('equilibre_prod', engine , index=False, if_exists='replace', chunksize=1000  )

In [507]:
sous_nutrition = pd.read_csv('fr_sousalimentation.csv')

In [508]:
sous_nutrition = sous_nutrition[['Zone', 'Code zone', 'Année', 'Valeur']]

In [509]:
sous_nutrition.rename(columns={
    'Zone': 'pays',
    'Code zone': 'code_pays',
    'Année': 'annee',
    'Valeur': 'nb_personnes'
}, inplace=True)

In [510]:
sous_nutrition.replace('2011-2013', '2012', inplace=True)
sous_nutrition.replace('2012-2014', '2013',  inplace=True)

In [511]:
sous_nutrition = sous_nutrition[ sous_nutrition.code_pays.isin(products.code_pays.unique()) ]

In [512]:
sous_nutrition.dropna(
    subset=['nb_personnes'],
    how='all',
    inplace=True
)

In [ ]:
Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

In [513]:
pd.read_sql_query("SELECT pays, sum(dispo_prot) AS dispo_prot_g_per_jour FROM dispo_alim WHERE annee=2013 GROUP BY pays ORDER BY dispo_prot_g_per_jour DESC LIMIT 10",engine)


,pays,dispo_prot_g_per_jour
0,Islande,133.06
1,Chine - RAS de Hong-Kong,129.07
2,Israël,128.00
3,Lituanie,124.36
4,Maldives,122.32
5,Finlande,117.56
6,Luxembourg,113.64
7,Monténégro,111.90
8,Pays-Bas,111.46
9,Albanie,111.37


In [ ]:
Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de kcalories

In [514]:
pd.read_sql_query("SELECT pays, annee, sum(dispo_alim_kcal_p_j) as dispo_alim_kcal_p_jour FROM dispo_alim GROUP BY pays, annee ORDER BY dispo_alim_kcal_p_jour DESC LIMIT 10",engine)

,pays,annee,dispo_alim_kcal_p_jour
0,Autriche,2013,3770.0
1,Belgique,2013,3737.0
2,Turquie,2013,3708.0
3,États-Unis d'Amérique,2013,3682.0
4,Israël,2013,3610.0
5,Irlande,2013,3602.0
6,Italie,2013,3578.0
7,Luxembourg,2013,3540.0
8,Égypte,2013,3518.0
9,Allemagne,2013,3503.0


In [ ]:
les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.

In [515]:
nombre = 365

In [516]:
pd.read_sql_query("SELECT pays, annee, sum(dispo_prot) AS dispo_prot_g_per_jour FROM dispo_alim WHERE annee = 2012 GROUP BY pays UNION SELECT pays, annee, sum(dispo_prot) AS dispo_prot_g_per_jou FROM dispo_alim WHERE annee = 2013 GROUP BY pays ORDER BY dispo_prot_g_per_jou ASC LIMIT 20", engine)


,pays,annee,dispo_prot_g_per_jour
0,Libéria,2013,37.66
1,Guinée-Bissau,2013,44.05
2,Mozambique,2013,45.68
3,République centrafricaine,2013,46.04
4,Madagascar,2013,46.69
5,Haïti,2013,47.70
6,Zimbabwe,2013,48.32
7,Congo,2013,51.41
8,Ouganda,2013,52.64
9,Sao Tomé-et-Principe,2013,53.10


In [ ]:
La quantité totale (en kg) de produits perdus par pays en 2013.

In [517]:
pd.read_sql_query("SELECT pays, annee, SUM(pertes) FROM equilibre_prod GROUP BY pays ORDER BY SUM(pertes) DESC ", engine)

,pays,annee,SUM(pertes)
0,"Chine, continentale",2013,89575.0
1,Brésil,2013,75914.0
2,Inde,2013,55930.0
3,Nigéria,2013,19854.0
4,Indonésie,2013,13081.0
...,...,...,...
169,Bermudes,2013,0.0
170,Islande,2013,0.0
171,Kiribati,2013,0.0
172,Maldives,2013,0.0


In [ ]:
Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

In [532]:
df_sous_nutrition = pd.read_csv('fr_sousalimentation.csv')
df_sous_nutrition.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20122014,2012-2014,millions,7.9,F,Estimation FAO,NaN
1,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20132015,2013-2015,millions,8.8,F,Estimation FAO,NaN
2,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20142016,2014-2016,millions,9.6,F,Estimation FAO,NaN
3,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20152017,2015-2017,millions,10.2,F,Estimation FAO,NaN
4,FS,Données de la sécurité alimentaire,2,Afghanistan,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20162018,2016-2018,millions,10.6,F,Estimation FAO,NaN


In [533]:
df_sous_nutrition = df_sous_nutrition[['Code zone','Zone','Code année','Année','Valeur']]

In [534]:
df_sous_nutrition = df_sous_nutrition.rename(columns={'Zone': "pays", 'Code zone':'code_pays', 'Valeur': 'nb_personnes'})

In [535]:
df_sous_nutrition.loc[df_sous_nutrition['Année'] =='2012-2014', 'Code année'] = '2013'
df_sous_nutrition.loc[df_sous_nutrition['Année'] =='2013-2015', 'Code année'] = '2014'
df_sous_nutrition.loc[df_sous_nutrition['Année'] =='2014-2016', 'Code année'] = '2015'
df_sous_nutrition.loc[df_sous_nutrition['Année'] =='2015-2017', 'Code année'] = '2016'

In [539]:
df_sous_nutrition =df_sous_nutrition.drop(columns = ['Année'])


In [538]:
df_sous_nutrition = df_sous_nutrition.rename(columns={'Code année': "Année"})


In [540]:
df_population =df_population[df_population.Année != 2017]
df_population['Année'] =df_population['Année'].astype(str)

In [541]:
df_sous_nutrition = df_sous_nutrition.merge(df_population)


In [542]:
df_sous_nutrition.to_sql('sous_nutrition', engine,  index=False, if_exists='replace')

In [543]:
pd.read_sql_table('sous_nutrition', engine)

,code_pays,pays,nb_personnes,Année,population
0,2,Afghanistan,7.9,2013,30552
1,2,Afghanistan,8.8,2013,30552
2,2,Afghanistan,9.6,2013,30552
3,2,Afghanistan,10.2,2013,30552
4,2,Afghanistan,10.6,2013,30552
...,...,...,...,...,...
870,181,Zimbabwe,6.6,2013,14150
871,181,Zimbabwe,7.2,2013,14150
872,181,Zimbabwe,7.8,2013,14150
873,181,Zimbabwe,8.2,2013,14150


In [544]:
#Requête pour toruver les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.
pd.read_sql_query("SELECT pays, Année, Sum(nb_personnes) FROM sous_nutrition WHERE Année = '2013' GROUP BY pays ORDER BY SUM(nb_personnes )DESC LIMIT 10", engine)

,pays,Année,Sum(nb_personnes)
0,Inde,2013,1020.7
1,Chine,2013,638.2
2,"Chine, continentale",2013,633.1
3,Pakistan,2013,196.0
4,Bangladesh,2013,126.1
5,Éthiopie,2013,115.3
6,Indonésie,2013,111.7
7,Nigéria,2013,93.0
8,République-Unie de Tanzanie,2013,84.9
9,Ouganda,2013,75.4


In [ ]:
Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [531]:
pd.read_sql_query("SELECT produit, AVG(autres_utilisations / dispo_int) AS ratio_autres_utilisations_dispo_int FROM equilibre_prod GROUP BY produit ORDER BY ratio_autres_utilisations_dispo_int DESC LIMIT 10",engine)

,produit,ratio_autres_utilisations_dispo_int
0,"Alcool, non Comestible",0.982456
1,Plantes Aquatiques,0.920661
2,Huile de Palmistes,0.773645
3,Piments,0.739130
4,Huile de Palme,0.652520
5,Huile de Colza&Moutarde,0.616865
6,Palmistes,0.575625
7,Huile de Coco,0.570894
8,Huil Plantes Oleif Autr,0.547179
9,Huile de Son de Riz,0.502955


In [ ]:
Question20

In [ ]:
plantes aquatiques  dans les aquariums pour lutter contre les algues

In [ ]:
Huile de palme pour la fabrication de savons  ou d' agrocarburant

In [ ]:
huile de coco  comme déodorant